# Exploratory Data Analysis

## About Dataset

- **`Description`**  
  The Data Set was downloaded from Kaggle, from the following [link](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data).

- **`Context`**  
  In real estate, a home's price is influenced by far more than just its size or number of rooms.  
  This dataset includes 81 detailed features of homes allowing a rich ground for exploring what really drives house prices.  
  It reveals how hidden patterns in data can explain price differences in homes.

- **`Content`**  
    This dataset contains 1,460 rows and 81 columns, with detailed information on residential properties in Ames, Iowa.  
  Features range from lot size, zoning, and neighborhood to quality ratings, year built, and sale conditions including the target variable, `SalePrice`.

- **`Acknowledgements`**  
    The dataset is part of a Kaggle competition and is built on the Ames Housing dataset by Dean De Cock.
     
- **`Inspiration`**  
  This dataset allows to explore what makes one home more valuable than another and how different features affect home prices. The insights can help people make smart choices in buying, selling, or investing in homes and support better decisions in the real estate market.


# Import the library

In [ ]:
import os
import seaborn as sns
import sys
from pathlib import Path
import numpy as np
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt 
import pandas as pd
import plotly.subplots as sp
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load dataset

In [ ]:
current_path = Path.cwd().resolve()
ROOT_DIR = current_path.parents[0]  # Get the parent directory of the current path
print(f"Root directory: {ROOT_DIR}")

DATA_DIR = ROOT_DIR / 'data'
print(f"Data directory: {DATA_DIR}")

TEST_DIR = ROOT_DIR / 'test'
print(f"Test directory: {TEST_DIR}")

In [ ]:
training_data = pd.read_csv(DATA_DIR / 'train.csv')

## Description of the shape and type of the dataset

In [ ]:
# Show shape of the dataset
print(f"Shape of the dataset: {training_data.shape}")

In [ ]:
# Show the columns of the dataset
print(f"Columns of the dataset: {training_data.columns.tolist()}")

In [ ]:
# Show the first few rows of the dataset
print(f"First few rows of the dataset: ")
training_data.head()

In [ ]:
# Show the description of the dataset
print(f"Description of the dataset: ")
training_data.describe().T

In [ ]:
# Drop the Id column
training_data = training_data.drop(columns=['Id']) 

In [ ]:
# Show information about the dataset
print(f"Information about the dataset: ")
training_data.info() 

In [ ]:
# Select unique values for object columns
object_df = training_data.select_dtypes(include='object')
object_cols = object_df.columns.unique().to_list()
print(f"Lenght of object columns: {len(object_cols)}")
print(f"Object columns and their unique values:")
object_df.nunique().to_dict()


In [ ]:
# select unique values for non-object columns
non_object_df = training_data.select_dtypes(exclude='object')
non_object_cols = non_object_df.columns.unique().to_list()
print(f"Lenght of non-object columns: {len(non_object_cols)}")
print(f"Non-object columns and their unique values:")
non_object_df.nunique().to_dict()

# Deal with missing values 

In [ ]:
def plot_missing_values(df):
    # Draw a heatmap using missing values
    missing_value = df.isnull().sum().sort_values(ascending=False)
    missing_value = missing_value[missing_value > 0]  # Filter out columns with no missing values
    if missing_value.empty:
        print("No missing values found in the dataset.")
        return None, None
    missing_value = missing_value.reset_index()

    # Create a bar chart using plotly
    fig_missing = px.bar(
        missing_value,  # Reshape to 2D array for heatmap
        labels={'index': 'Feature', '0': 'Missing Values'},
        x=missing_value['index'],
        y=missing_value[0],
        color_continuous_scale='Blues',
    ).update_layout(
        title='Missing Values Bar Chart',
        xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    ).show()

    return missing_value, fig_missing

In [ ]:
def plot_missing_values_percentage(df):
    # Calculate the percentage of missing values in each column
    missing_values = df.isnull().sum().sort_values(ascending=False)
    missing_values = missing_values[missing_values > 0]  # Filter out columns with no missing values
    if missing_values.empty:
        print("No missing values found in the dataset.")
        return None, None
    missing_values_percentage = missing_values / len(df) * 100

    # Create a bar chart using plotly
    fig_missing_percentage = px.bar(
        missing_values_percentage,  # Filter out columns with no missing values
        labels={'index': 'Feature', 'value': 'Missing Values Percentage'},
        x=missing_values_percentage.index,
        y=missing_values_percentage.values,
        color_continuous_scale='Blues',
    ).update_layout(
        title='Missing Values Percentage Bar Chart',
        xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    ).show()

    return missing_values_percentage, fig_missing_percentage

## Categorial columns

In [ ]:
plot_missing_values(object_df)

In [ ]:
plot_missing_values_percentage(object_df)

In [ ]:
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
training_data = training_data.drop(columns=cols_to_drop)

### 1. MasVnrType


In [ ]:
# Show sum of missing values of MasVnrType
mas_vnr_type_missing = training_data['MasVnrType'].isnull().sum()
print(f"Missing values in MasVnrType: {mas_vnr_type_missing}")

# Show the unique values of MasVnrType
mas_vnr_type_unique = training_data['MasVnrType'].unique()
print(f"Unique values in MasVnrType: {mas_vnr_type_unique}")

In [ ]:
# Because the MasVnrType column has a lot of missing values
# The value nan of MasVnrType column likely represents houses
# without masonry veneer, so we can fill it with 'None'
training_data['MasVnrType'] = training_data['MasVnrType'].fillna('None')

### 2. FirePlaceQu

In [ ]:
# Show the sum of missing values of FirePlaceQu
fireplace_qu_missing = training_data['FireplaceQu'].isnull().sum()
print(f"Missing values in FirePlaceQu: {fireplace_qu_missing}")

# Show the unique values of FirePlaceQu
fireplace_qu_unique = training_data['FireplaceQu'].unique()
print(f"Unique values in FirePlaceQu: {fireplace_qu_unique}")

In [ ]:
# Because the FirePlaceQu column has a lot of missing values
# The value nan of FirePlaceQu column represents fire place in houses 
# So we can fill it with 'None'
training_data['FireplaceQu'] = training_data['FireplaceQu'].fillna('None')

### 3. Garage: GarageType, GarageCond, GarageQual, GarageFinish

In [ ]:
# The GarageType, GarageCond, GarageQual, GarageFinish columns have a some missing values
# We can fill them with 'None' because they represent houses without garage
garage_cols = ['GarageType', 'GarageCond', 'GarageQual', 'GarageFinish']
for col in garage_cols:
    training_data[col] = training_data[col].fillna('None')

### 4. Basement:  BsmtFinType2, BsmtExposure, BsmtFinType1, BsmtQual, BsmtCond 

In [ ]:
# The basement columns have a some missing values
# We can fill them with 'None' because they represent houses without basement
basement_cols = ['BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond']
for col in basement_cols:
    training_data[col] = training_data[col].fillna('None')

### 5. Electrical

In [ ]:
# Show the missing values of Electrical
electrical_missing = training_data['Electrical'].isnull().sum()
print(f"Missing values in Electrical: {electrical_missing}")    

# Show the unique values of Electrical
electrical_unique = training_data['Electrical'].unique()
print(f"Unique values in Electrical: {electrical_unique}")

In [ ]:
# The Electrical column has a just one missing value
# We can fill it with the most common value in the column
most_common_electrical = training_data['Electrical'].mode()[0]
print(f"Most common value in Electrical: {most_common_electrical}")

# Fill the missing value in Electrical with the most common value
training_data['Electrical'] = training_data['Electrical'].fillna(most_common_electrical)

### Check the missing value of object columns

In [ ]:
# Drop the columns that have been removed from object_cols
for i in cols_to_drop:
    try:
        object_cols.remove(i)
    except ValueError:
        print(f"Column {i} not found in object_cols, skipping removal.")

In [ ]:
# Check the missing value of object columns
object_df = training_data.select_dtypes(include='object')[object_cols]
plot_missing_values(object_df)


## Numerical columns

In [ ]:
plot_missing_values(non_object_df)

In [ ]:
plot_missing_values_percentage(non_object_df)

### 1. LotFrontage

In [ ]:
# Show the sum of missing values in LotFrontage
lot_frontage_missing = training_data['LotFrontage'].isnull().sum()
print(f"Sum of missing values in LotFrontage: {lot_frontage_missing}")

# Check the unique values in LotFrontage
unique_lot_frontage = training_data['LotFrontage'].unique()
print(f"Unique values in LotFrontage: {unique_lot_frontage}")

In [ ]:
# plot the violin plot for LotFrontage
px.violin(training_data, y='LotFrontage', box=True, points='all')

In [ ]:
# The LotFrontage column has some outliers and missing values.
# We can filled the missing values in LotFrontage with the mean value
mean_lot_frontage = training_data['LotFrontage'].mean()
# Fill the missing values in LotFrontage with the mean value
training_data['LotFrontage'] = training_data['LotFrontage'].fillna(mean_lot_frontage)

### 2. GarageYrBlt

In [ ]:
# Get index of nan value of GarageType
index_nan_GarageType = training_data[training_data['GarageType'] == 'None'].index

# Get index of nan value of GaraYrBlt
index_nan_GaraYrBlt = training_data[training_data['GarageYrBlt'].isna()].index

# Compare index of 2 features
index_nan_GarageType == index_nan_GaraYrBlt

In [ ]:
# Because the nan index of GarageType and GaraYrBlt is same
# So we can't use median or mean value to fill in the nan value (These homes 
# don't have a garage)
training_data['GarageYrBlt'] = training_data['GarageYrBlt'].fillna(0)

### 3. MasVnrArea

In [ ]:
# 1. Check how many rows have missing values (NaN) in 'MasVnrArea'
print("Number of rows with MasVnrArea as NaN:", training_data[training_data['MasVnrArea'].isnull()].shape[0])  # Output: 8

In [ ]:
# 2. Display details of these rows to inspect their 'MasVnrType'
print("\nDetails of rows where MasVnrArea is NaN:")
display(training_data[training_data['MasVnrArea'].isnull()][['MasVnrType', 'MasVnrArea']])

In [ ]:
# 3. Count how many houses have no masonry veneer (MasVnrType == 'None')
print(f"\nNumber of houses with MasVnrType = 'None': {training_data[training_data['MasVnrType'] == 'None'].shape[0]}")  # Output: 872

In [ ]:
# # 4. Find inconsistent rows: MasVnrType is 'None' but MasVnrArea is not zero
# # This violates logical consistency — if there's no veneer, area should be 0
inconsistent_rows = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0)]
print(f"\nNumber of inconsistent rows (MasVnrType='None' but MasVnrArea ≠ 0): {len(inconsistent_rows)}")
inconsistent_rows[['MasVnrType', 'MasVnrArea']]

In [ ]:
# # 5. Check consistent rows: MasVnrType is 'None' and MasVnrArea is 0
# # These are logically correct
consistent_rows = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] == 0)]
print(f"Number of consistent rows (MasVnrType='None' and MasVnrArea = 0): {len(consistent_rows)}")

In [ ]:
# # 6. FIX INCONSISTENCY: Set MasVnrArea = 0 where MasVnrType is 'None' but area is non-zero
# # Logical rule: No veneer → area must be zero
training_data.loc[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0), 'MasVnrArea'] = 0

In [ ]:
# 7. HANDLE MISSING VALUES in MasVnrArea
# We now handle the 8 NaNs in MasVnrArea based on MasVnrType:
# - If MasVnrType is 'None', then MasVnrArea should be 0
# - Otherwise, impute using group mean (by MasVnrType) for better accuracy

# Step 1: Fill MasVnrArea = 0 if MasVnrType is 'None' and MasVnrArea is NaN
training_data.loc[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'].isnull()), 'MasVnrArea'] = 0

# Step 2: For remaining NaNs, fill with the average MasVnrArea within the same MasVnrType group
# This preserves patterns — e.g., 'BrkFace' homes will use average BrkFace area
training_data['MasVnrArea'] = training_data.groupby('MasVnrType')['MasVnrArea'].transform(lambda x: x.fillna(x.mean()))

# # Step 3: If any NaNs still remain (e.g., a group had all NaNs), fill with overall mean
if training_data['MasVnrArea'].isnull().any():
    training_data['MasVnrArea'] = training_data['MasVnrArea'].fillna(training_data['MasVnrArea'].mean())

# Step 4 Final validation: Ensure no inconsistencies remain
# Double-check that no house has 'None' veneer type but non-zero area
final_check = training_data[(training_data['MasVnrType'] == 'None') & (training_data['MasVnrArea'] != 0)]
print(f"Length of final check: {final_check}")

# Descriptive Statistic

Although `MSSubClass` represents **categorical codes for types of dwellings** involved in the sale. Description in 'data_description.txt':

    20	1-STORY 1946 & NEWER ALL STYLES
    30	1-STORY 1945 & OLDER
    40	1-STORY W/FINISHED ATTIC ALL AGES
    45	1-1/2 STORY - UNFINISHED ALL AGES
    50	1-1/2 STORY FINISHED ALL AGES
    60	2-STORY 1946 & NEWER
    70	2-STORY 1945 & OLDER
    75	2-1/2 STORY ALL AGES
    80	SPLIT OR MULTI-LEVEL
    85	SPLIT FOYER
    90	DUPLEX - ALL STYLES AND AGES
    120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
    150	1-1/2 STORY PUD - ALL AGES
    160	2-STORY PUD - 1946 & NEWER
    180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
    190	2 FAMILY CONVERSION - ALL STYLES AND AGES

-> we will convert it to categorical

In [ ]:
training_data['MSSubClass'] = training_data['MSSubClass'].astype('object')
object_cols = training_data.select_dtypes(include= 'object')

In [ ]:
'MSSubClass' in object_cols.columns

## Histogram

In [ ]:
# Create subplot gird for all numerical features
rows = (len(non_object_df.columns) + 3) // 4
fig = sp.make_subplots(
    rows= rows,
    cols= 4,
    subplot_titles= non_object_df.columns
) 

for i, col in enumerate(non_object_df.columns):
    row = i // 4 + 1
    col_pos = i % 4 + 1
    fig.add_trace(
        go.Histogram(x= non_object_df[col], name= col),
        row= row, col= col_pos
    )

fig.update_layout(
    title_text="Histograms for Numerical Columns",
    height=300*rows,
    showlegend=False
)
fig.show()


- Most features like house area and garage size have small values for most homes, but a few are very big.

- Features like number of bathrooms, garages, and fireplaces have specific counts, mostly 1, 2, or 3.

- Year-related features show that more houses were built or remodeled in recent years.

- Some features like LotArea, PoolArea, and MiscVal have a few very high values.

- The house price (SalePrice) is also higher for a few homes, but most are mid-range.

## Correlation matrix

In [ ]:
# Calculate the correlation matrix for non-object columns
coor_matrix = training_data[non_object_cols].corr()
coor_matrix

In [ ]:
# Draw the heatmap using plotly
# Note: Plotly does not have a direct equivalent of seaborn's heatmap, but we can use px.imshow to create a similar effect
fig = px.imshow(
    coor_matrix,
    text_auto='.2f',                # Display correlation values with 2 decimal places
    color_continuous_scale='RdBu_r', # Set color scale
    aspect='auto',               # Adjust aspect ratio
    labels=dict(color="Correlation")
)

# Configure the layout of the heatmap
fig.update_layout(
    title='Correlation Heatmap',
    width=1200,
    height=1200,
    xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    yaxis=dict(tickangle=0)
)

# Hiển thị biểu đồ
fig.show()

Skewness describes the shape of a distribution and tells us if the data is symmetrical or not.

Skewness = 0 → symmetrical 

Skewness > 0 → right (positive) skew 

Skewness < 0 → left (negative) skew

In [ ]:
# Get the skweness of the dataset
skewness = training_data[non_object_cols].skew().sort_values(ascending=False)
print(f"Skewness of the dataset: {skewness}")
print('\n')
print(f"5 highest skewness values: \n{skewness.head(5)}")

Kurtosis ≈ 0: The distribution has a sharpness similar to the normal distribution (called mesokurtic).

Kurtosis > 0: The distribution is more peaked than normal, with a higher peak and heavier tails (leptokurtic).

→ Data is highly concentrated around the mean, but has more extreme values (outliers) in the tails.

Kurtosis < 0: The distribution is flatter than normal, with a lower peak and thinner tails (platykurtic).

→ Data is more evenly spread out and less concentrated around the center.

In [ ]:
# Get the kurtosis of the dataset
kurtosis = training_data[non_object_cols].kurtosis().sort_values(ascending=False)
print(f"Kurtosis of the dataset: {kurtosis}")
print('\n')
print(f"5 highest kurtosis values: \n{kurtosis.head(5)}")

In [ ]:
# Get 10 highest correlated features with SalePrice
list_highest_coor = list(coor_matrix['SalePrice'].abs().sort_values(ascending=False).head(10).to_dict())
print(f"10 highest correlated features with SalePrice: {list_highest_coor}")

# Get coorrelation matrix for the highest correlated features
coor_matrix_highest = training_data[list_highest_coor].corr()

In [ ]:
# Draw the heatmap using plotly
# Note: Plotly does not have a direct equivalent of seaborn's heatmap, but we can use px.imshow to create a similar effect
fig = px.imshow(
    coor_matrix_highest,
    text_auto='.2f',                # Display correlation values with 2 decimal places
    color_continuous_scale='RdBu_r', # Set color scale
    aspect='auto',               # Adjust aspect ratio
    labels=dict(color="Correlation")
)

# Configure the layout of the heatmap
fig.update_layout(
    title='Correlation Heatmap',
    width=800,
    height=800,
    xaxis=dict(tickangle=-45),      # Rotate x-axis labels 
    yaxis=dict(tickangle=0)
)

# Hiển thị biểu đồ
fig.show()

Top Features Most Correlated with SalePrice

OverallQual → 0.79: The strongest correlated feature with sale price, reflecting the overall quality of materials and finish of the house.

GrLivArea → 0.71: Above-ground living area; larger homes typically sell for higher prices.

GarageCars → 0.64: Number of cars that can fit in the garage; more garage spaces generally increase the home’s value.

GarageArea → 0.62: Total garage area (sq ft); larger garages are usually associated with higher sale prices.

TotalBsmtSF → 0.61: Total basement area; bigger basements tend to increase the selling price.

1stFlrSF → 0.61: First floor square footage; a larger first floor is linked to higher home values.

FullBath → 0.56: Number of full bathrooms; more full bathrooms typically result in a higher sale price.

TotRmsAbvGrd → 0.53: Total number of rooms above ground; more rooms generally correlate with a higher selling price.

YearBuilt → 0.52: Year the house was built; newer houses tend to sell for higher prices.

Note: Several features are highly correlated with each other, which can lead to multicollinearity. Examples include: GarageCars & GarageArea or TotalBsmtSF & 1stFlrSF

## Top 10 Features Most Correlated with SalePrice|

### OverallQual

-> `OverallQual` has the **strongest correlation** with `SalePrice` at **0.79**.
- As the quality rating increases, the house price also increases.
- It shows the overall material and finish quality, with values from **1 (Very Poor)** to **10 (Excellent)**.

In [ ]:
# Box plot for OverallQual
px.box(
    data_frame= training_data, 
    x = 'OverallQual',
    y = 'SalePrice', 
    title= 'Sale price distribution by overall quality',
).show()


### GrLivArea

In [ ]:
# Box plot for GrLivArea
px.scatter(
    data_frame= training_data, 
    x = 'GrLivArea',
    y = 'SalePrice', 
    color= 'OverallQual',
    title= 'Sale price distribution by overall quality',
).show()


There are some point in the ground living area more than 4000 square feet with overall quality 10 is cost

less than 400k $. Such points are outliers. So we must remove these points. 



In [ ]:
outlier_mask = (training_data['GrLivArea'] > 4000) & (training_data['SalePrice'] < 200000) & (training_data['OverallQual'] == 10)
training_data = training_data[~outlier_mask]

### GarageCars

In [ ]:
# Box plot for GarageCars
px.box(
    data_frame= training_data, 
    x = 'GarageCars',
    y = 'SalePrice', 
    title= 'Sale price distribution by garage car',
).show()

Sale price increase with garage size, and there are some outliers for 3-cars. 

### GarageArea 


In [ ]:
# Box plot for GarageCars
px.box(
    data_frame= training_data, 
    x = 'GarageArea',
    title= 'Box plot garage area',
).show()

The common areas in range [330, 570] square feet

Have outliers on the garage areas greater than ~950 square ft.

### 

In [ ]:
# Histogram of garage areas
px.histogram(
    data_frame= training_data,
    x = 'GarageArea',
    title= 'Distribution garage areas',
).show()

Distribution of garage area is right positively skewed.

In [ ]:
# Scatter plot 
px.scatter(
    data_frame= training_data,
    x = 'GarageArea',
    y = 'SalePrice',
    color= 'OverallQual'
).show()

As the garage area increases, the sale price generally tends to increase as well.

The relationship is not perfectly linear, at larger garage areas there is spread.

### TotalBsmtSF

In [ ]:
# Box plot for TotalBsmtSF
px.box(
    data_frame= training_data, 
    x = 'TotalBsmtSF',
    title= 'Box plot TotalBsmtSF',
).show()

The common total square feet of basement area in range [330, 570] square feet

Have outliers on the garage areas greater than ~2050 square ft.

In [ ]:
# Histogram of TotalBsmtSF
px.histogram(
    data_frame= training_data,
    x = 'TotalBsmtSF',
    title= 'Distribution total square feet of basement area',
).show()

Total basement area distribution is right-skewed (positively skewed).

Most homes have a basement area between 800 and 1800 square feet, with a peak around 1000 square feet.

There are a little homes with very large basement areas (greater than 3000 sq ft).


In [ ]:
# Scatter plot 
px.scatter(
    data_frame= training_data,
    x = 'TotalBsmtSF',
    y = 'SalePrice',
    color= 'OverallQual'
).show()

### 1stFlrSF

In [ ]:
# Box plot for 1stFlrSF
px.box(
    data_frame= training_data, 
    x = '1stFlrSF',
    title= 'Box plot 1stFlrSF',
).show()

In [ ]:
# Histogram of 1stFlrSF
px.histogram(
    data_frame= training_data,
    x = '1stFlrSF',
    title= 'Distribution first floor square feet',
).show()

In [ ]:
# Scatter plot 
px.scatter(
    data_frame= training_data,
    x = '1stFlrSF',
    y = 'SalePrice',
    color= 'OverallQual'
).show()

### FullBath 

In [ ]:
# Box plot for FullBath
px.box(
    data_frame= training_data, 
    x = 'FullBath',
    title= 'Box plot FullBath',
).show()

Full Bath has no outliers

In [ ]:
# Histogram of 1stFlrSF
px.histogram(
    data_frame= training_data,
    x = 'FullBath',
    title= 'Distribution of full bathrooms above grade',
).show()

In [ ]:
# Scatter plot 
px.scatter(
    data_frame= training_data,
    x = 'FullBath',
    y = 'SalePrice',
    color= 'OverallQual'
).show()

### YearBuilt

In [ ]:
# Box plot for FullBath
px.box(
    data_frame= training_data, 
    x = 'YearBuilt',
    title= 'Box plot YearBuilt',
).show()

There are few houses built before 1885.  

In [ ]:
# Histogram of 1stFlrSF
px.histogram(
    data_frame= training_data,
    x = 'YearBuilt',
    title= 'Distribution of original construction date',
).show()

In [ ]:
# Scatter plot 
px.scatter(
    data_frame= training_data,
    x = 'YearBuilt',
    y = 'SalePrice',
    color= 'OverallQual'
).show()

In this plot, new house have a overall quality more than the older. 

But from that, the sale price is weakly affected by year built.

## Sale price 

### Draw a histogram and violinplot

In [ ]:
# Use plotly to visualize the distribution of SalePrice
px.histogram(data_frame= training_data, x= 'SalePrice', title='Distribution of SalePrice')

In [ ]:
# Plot the violin plot for SalePrice
px.violin(training_data, 
        y='SalePrice',
        box=True, points='all', 
        title='Distribution of SalePrice', 
        width=800,
height=800)

In [ ]:
# Skewness and kurtosis
print(f"Skewness of SalePrice: {training_data['SalePrice'].skew()}")
print(f"Kurtosis of SalePrice: {training_data['SalePrice'].kurtosis()} ")

### Draw distribution analysis and Q-Q plot

In [ ]:
def plot_hist_pdf(feature_name, data_frame):
    # Create histogram with normal probability density function overlay using seaborn
    
    plt.figure(figsize=(10, 6))
    
    # Create histogram with density and KDE
    sns.histplot(data_frame[feature_name], kde=True, stat='density', alpha=0.7)
    
    # Fit normal distribution and overlay
    mu, std = norm.fit(data_frame[feature_name].dropna())
    x = np.linspace(data_frame[feature_name].min(), data_frame[feature_name].max(), 100)
    plt.plot(x, norm.pdf(x, mu, std), 'r-', linewidth=2, 
             label=f'Normal Fit (μ={mu:.2f}, σ={std:.2f})')
    
    plt.title(f'Histogram of {feature_name} with Normal Fit')
    plt.xlabel(feature_name)
    plt.ylabel('Density')
    plt.legend()
    plt.show()

def plot_qq_plot(data_frame, column_name, title="Q-Q Plot"):
    # Create a Q-Q plot to test for normality of data using scipy and matplotlib
    
    plt.figure(figsize=(8, 6))
    
    # Create Q-Q plot
    stats.probplot(data_frame[column_name].dropna(), dist="norm", plot=plt)
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

def apply_log_transform(feature_name, data_frame):
    # Apply log1p transformation to make distribution more normal
    # Create a copy to avoid modifying original data
    df_copy = data_frame.copy()
    df_copy[feature_name] = np.log1p(data_frame[feature_name])
    return df_copy

def analyze_distribution(data_frame, feature_name, apply_log=False):    
    # Complete distribution analysis with before/after log transformation
    print(f"=== Distribution Analysis for {feature_name} ===")
    
    # Original distribution
    print("\n1. Original Distribution:")
    
    # Histogram with normal fit
    plt.figure(figsize=(15, 5))
    
    # Subplot 1: Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(data_frame[feature_name], kde=True, stat='density', alpha=0.7)
    mu, std = norm.fit(data_frame[feature_name].dropna())
    x = np.linspace(data_frame[feature_name].min(), data_frame[feature_name].max(), 100)
    plt.plot(x, norm.pdf(x, mu, std), 'r-', linewidth=2, 
             label=f'Normal Fit (μ={mu:.2f}, σ={std:.2f})')
    plt.title(f'Original {feature_name} Distribution')
    plt.xlabel(feature_name)
    plt.ylabel('Density')
    plt.legend()
    
    # Subplot 2: Q-Q Plot
    plt.subplot(1, 2, 2)
    stats.probplot(data_frame[feature_name].dropna(), dist="norm", plot=plt)
    plt.title(f'Q-Q Plot: Original {feature_name}')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    if apply_log:
        # Apply log transformation
        df_transformed = apply_log_transform(feature_name, data_frame)
        
        print(f"\n2. After Log Transformation:")
        
        # Log-transformed distribution plots
        plt.figure(figsize=(15, 5))
        
        # Subplot 1: Log-transformed histogram
        plt.subplot(1, 2, 1)
        sns.histplot(df_transformed[feature_name], kde=True, stat='density', alpha=0.7)
        mu_log, std_log = norm.fit(df_transformed[feature_name].dropna())
        x_log = np.linspace(df_transformed[feature_name].min(), df_transformed[feature_name].max(), 100)
        plt.plot(x_log, norm.pdf(x_log, mu_log, std_log), 'r-', linewidth=2,
                 label=f'Normal Fit (μ={mu_log:.2f}, σ={std_log:.2f})')
        plt.title(f'Log-transformed {feature_name} Distribution')
        plt.xlabel(f'log1p({feature_name})')
        plt.ylabel('Density')
        plt.legend()
        
        # Subplot 2: Log-transformed Q-Q Plot
        plt.subplot(1, 2, 2)
        stats.probplot(df_transformed[feature_name].dropna(), dist="norm", plot=plt)
        plt.title(f'Q-Q Plot: Log-transformed {feature_name}')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return df_transformed
    
    return data_frame

In [ ]:
cols_to_log = ['SalePrice', 'LotArea', '1stFlrSF', 'GrLivArea', 'TotalBsmtSF']

for col in cols_to_log:
    training_data = analyze_distribution(training_data, col, True)


### Draw boxplots of categorical columns

In [ ]:
def create_categorical_boxplots(df, categorical_columns, target_col='SalePrice', cols=2):
    # Create boxplots for categorical variables vs target variable using Plotly subplots
    
    # Calculate number of rows needed
    n_vars = len(categorical_columns)
    rows = (n_vars + cols - 1) // cols  # Ceiling division
    
    # Create subplot titles
    subplot_titles = categorical_columns
    
    # Create subplots
    fig = make_subplots(
        rows=rows, 
        cols=cols,
        subplot_titles=subplot_titles,
        vertical_spacing=0.08,
        horizontal_spacing=0.05
    )
    
    # Color palette (similar to tab10)
    colors = px.colors.qualitative.Set1
    
    for i, var in enumerate(categorical_columns):
        row = i // cols + 1
        col = i % cols + 1
        
        # Get unique categories and sort them
        categories = sorted(df[var].dropna().unique())
        
        # Create boxplot for each category
        for j, category in enumerate(categories):
            data_subset = df[df[var] == category][target_col].dropna()
            
            fig.add_trace(
                go.Box(
                    y=data_subset,
                    name=str(category),
                    boxpoints='outliers',
                    marker_color=colors[j % len(colors)],
                    showlegend=False
                ),
                row=row, col=col
            )
        
        # Update x-axis for this subplot
        fig.update_xaxes(
            title_text=var,
            tickangle=45,
            row=row, col=col
        )
        
        # Update y-axis for this subplot
        fig.update_yaxes(
            title_text=target_col if col == 1 else "",
            row=row, col=col
        )
    
    # Update overall layout
    fig.update_layout(
        height=400 * rows,
        width=1200,
        title_text=f"Boxplots of {target_col} by Categorical Variables",
        title_x=0.5,
        showlegend=False
    )
    
    fig.show()
    return fig

def single_boxplot_plotly(df, x_col, y_col, title=None):
    # Create a single boxplot using Plotly
    # Method 3: Single boxplot
    # fig3 = single_boxplot_plotly(df, 'Neighborhood', 'SalePrice')
    fig = px.box(
        df, 
        x=x_col, 
        y=y_col,
        color=x_col,
        title=title or f"{y_col} by {x_col}"
    )
    
    fig.update_xaxes(tickangle=45)
    fig.update_layout(
        showlegend=False,
        width=800,
        height=500
    )
    
    fig.show()
    return fig

In [ ]:
# Get categorical column names
categorical_cols = object_df.columns.tolist()
print(f"Name of categorical columns: {categorical_cols}")


In [ ]:
# plot the categorical boxplots
fig1 = create_categorical_boxplots(training_data, categorical_cols, 'SalePrice', cols=5)

The boxplots show that some categories, like Zoning, Neighborhood, and Exterior quality, have a strong effect on house prices. For example, certain neighborhoods and better exterior quality are linked to higher prices.

Features like central air also tend to raise prices.

Some variables, such as Street type and Utilities, do not make much difference in price.

### Draw median bar plot a numerical columms

In [ ]:
# Check number of unique values of numerical columns 

for i in non_object_cols:
    print(f"Number of unique value of {i}: {len(training_data[i].unique())}")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

threshold = 15
discrete_features = [
    col for col in non_object_cols
    if training_data[col].nunique() <= threshold and col != 'SalePrice'
]

n_cols = 3
n_rows = math.ceil(len(discrete_features) / n_cols)

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=discrete_features
)

# Chuẩn bị dữ liệu median cho từng feature
grouped_data = {
    feature: training_data.groupby(feature)['SalePrice'].median().reset_index()
    for feature in discrete_features
}

for i, (feature, grouped) in enumerate(grouped_data.items()):
    r, c = divmod(i, n_cols)
    fig.add_trace(
        go.Bar(
            x=grouped[feature],
            y=grouped['SalePrice'],
            marker_color=grouped['SalePrice'],
            marker_colorscale='Viridis',
            name=feature
        ),
        row=r+1, col=c+1
    )
    fig.update_xaxes(title_text=feature, row=r+1, col=c+1)
    fig.update_yaxes(title_text="Median SalePrice", row=r+1, col=c+1)

fig.update_layout(
    height=350 * n_rows,
    width=1400,
    title_text="Median SalePrice by Discrete Numerical Features",
    showlegend=False
)

fig.show()


1. **OverallQual (Overall Quality):**

   * The most influential feature on house prices.
   * Houses with high-quality ratings (9–10) have significantly higher median sale prices compared to those with low ratings (1–3), showing a strong positive linear relationship.

2. **OverallCond (Overall Condition):**

   * Sale prices remain relatively stable across most condition levels.
   * Only at the highest condition levels (8–9) do prices slightly increase, suggesting condition is less impactful than quality.

3. **BsmtFullBath and BsmtHalfBath (Basement Bathrooms):**

   * Having additional full or half bathrooms in the basement increases sale prices.
   * The effect is moderate compared to stronger factors like quality or garage size.

4. **FullBath and HalfBath (Main Bathrooms):**

   * Important contributors to pricing: more full bathrooms strongly correlate with higher prices.
   * Adding a half bathroom also raises value, but to a lesser extent than full bathrooms.

5. **BedroomAbvGr (Bedrooms) and KitchenAbvGr (Kitchens):**

   * The number of bedrooms has little impact on sale price; beyond a certain point, extra bedrooms do not add significant value.
   * Most houses have a single kitchen; having multiple kitchens does not increase prices and may even be linked to lower valuations.

6. **TotRmsAbvGrd (Total Rooms Above Ground):**

   * Shows a positive correlation with sale price: houses with more rooms usually have larger floor areas and higher prices.

7. **Fireplaces:**

   * More fireplaces are associated with higher prices, reflecting added comfort and luxury features.

8. **GarageCars (Garage Capacity):**

   * One of the strongest predictors of price.
   * Homes with 3–4 garage spaces have substantially higher median sale prices than those with just one.

9. **PoolArea:**

   * Pools are rare, but when present, they significantly increase sale prices.
   * This feature contains outliers and should be treated carefully during modeling.

10. **MoSold (Month Sold) and YrSold (Year Sold):**

    * The month of sale has negligible impact on pricing.
    * The year of sale introduces only minor variations (e.g., slightly higher prices in 2009), not a decisive factor.
